# PowerShell Cloud Connections

### Azure

Summary | Access and Manage Azure Resources (ARM)
--|--
Module | Az.Accounts (part of Az)
Permissions | RBAC assigned within Azure Tenant: Management Group, Subscription, Resource Group, or Resource
Reference | [Link](https://learn.microsoft.com/en-us/powershell/module/az.accounts/connect-azaccount?view=azps-10.4.1)

In [ ]:
Get-Help Connect-AzAccount

#### Connecting to Azure

In [ ]:
$conn = Connect-AzAccount

Disconnect-AzAccount

In [ ]:
$conn | Format-List

In [ ]:
Get-AzContext | Select Name,Account,Environment,Subscription | Format-List

In [ ]:
if (-not $conn) {
	Write-Host "Connecting to Azure..."
	$global:conn = Connect-AzAccount
} else {
	Write-Host "Already connected to Azure."
}

Connecting from within an Automation Runbook (assuming the Automation Account has a system-assigned Identity, and granted API permissions)

In [ ]:
$conn = Connect-AzAccount -Identity

#### Connection Context

Each time you establish a new Azure connection, the "context" of that connection is retained.
* You can accumulate multiple Contexts
* Each Context will exist until the cached token expires
* You can switch between connections by Context without having to re-authenticate

In [ ]:
$currentContext = Get-AzContext
$currentContext | Select-Object Subscription,@{l='Name';e={$($_.Name.Substring(0,52))}},Account | FL

Changing Context

In [ ]:
# list all subscriptions in the current tenant (based on RBAC access)
Get-AzSubscription | Select-Object Name,Id

In [ ]:
# change to subscription "Contoso"
Set-AzContext "Contoso"

In [ ]:
# change back to previous subscription
Set-AzContext $currentContext

### Microsoft Graph

Summary | Access and Manage Azure AD and Intune environments and related objects
--|--
Note | *Replaces the AzureAD and AzureADPreview modules*
Module | Microsoft.Graph and Microsoft.Graph.Authentication
Permissions | RBAC assigned within Teams or EntraID
Reference | [Link](https://learn.microsoft.com/en-us/powershell/module/teams/connect-microsoftteams?view=teams-ps)

In [ ]:
Connect-MgGraph -NoWelcome -TenantId $tenantID
$users = Get-MgUser -ConsistencyLevel eventual -All
Write-Host "$($users.Count) users returned"

In [ ]:
$users[4] | Select-Object *

### SharePoint Online

Summary | Access and Manage SharePoint Online Sites, Lists, and Much More. Can also connect to the management service.
--|--
Module | pnp.PowerShell
Permissions | RBAC assigned within SharePoint Online or EntraID
Reference | [Link](https://pnp.github.io/powershell/cmdlets/Connect-PnPOnline.html)

In [ ]:
$MyURL = 'https://skatterbrainz.sharepoint.com/sites/UserManagement'
$conn = Connect-PnPOnline -Url $MyURL -ReturnConnection -Interactive
$conn

### Exchange Online

Summary | Access and Manage Exchange Online Organization, Mailboxes, and related Settings
--|--
Module | ExchangeOnlineManagement

In [ ]:
$params = @{
	Credential = $Credential
	ExchangeEnvironmentName = "O365Default"
	Organization = "contoso.onmicrosoft.com"
	ShowBanner   = $False
	ShowProgress = $False
	ErrorAction  = 'Stop'
}

Connect-ExchangeOnline @params

# (do something here)

Disconnect-ExchangeOnline -Confirm:$False -ErrorAction SilentlyContinue

In [ ]:
$params = @{
	AppId = $Credential.UserName 
	CertificateThumbprint   = $Certificate.Thumbprint
	ExchangeEnvironmentName = "O365Default"
	Organization = "contoso.onmicrosoft.com"
	ShowBanner   = $False
	ShowProgress = $False
	ErrorAction  = 'Stop'
}

Connect-ExchangeOnline @params

# (do something here)

$null = Disconnect-ExchangeOnline -Confirm:$False -ErrorAction SilentlyContinue

In [ ]:
Connect-ExchangeOnline -Identity

# (do something here)

$null = Disconnect-ExchangeOnline -Confirm:$False -ErrorAction SilentlyContinue

### Exchange Security and Compliance

In [ ]:
$params = @{}

Connect-IPPSSession @params

### Exchange On-Premises

Summary | Access and Manage On-Prem Exchange Instances
--|--
Module | (Imported on Demand)
Permissions | RBAC assigned within Exchange
Reference | [Link](https://learn.microsoft.com/en-us/powershell/exchange/connect-to-exchange-servers-using-remote-powershell?view=exchange-ps)

In [ ]:
$params = @{
	ConfigurationName = "Microsoft.Exchange"
	ConnectionUri     = "http://mailserver03/powershell"
	Credential        = $Credential
}
$ExchSession = New-PSSession @params
$null = Import-PSSession -Session $ExchSession -DisableNameChecking -AllowClobber

# (do something here)

$null = Remove-PSSession $ExchSession

### Teams

Summary | Access and Manage Microsoft Teams and Team Channels
--|--
Module | MicrosoftTeams

In [ ]:
Connect-MicrosoftTeams

In [ ]:
$teams = Get-Team
$teams[0] | Select *

In [ ]:
$params = @{
	DisplayName = "Project Bluefish"
	Description = "Project Bluefish Development and Collaboration"
	Owner       = "dave@skatterbrainz.xyz"
	Visibility  = "Public"
}
$team = New-Team @params

In [ ]:
$params = @{
	GroupId = $team.GroupId
	User    = "emily@skatterbrainz.xyz"
	Role    = "Member"
}
Add-TeamUser @params